In [ ]:
import random
import smtplib
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Se importan las siguientes librerías:
# - `random`: para aleatorizar la asignación de amigos invisibles.
# - `smtplib`: para enviar correos electrónicos a través del protocolo SMTP.
# - `pandas`: para manejar el archivo de participantes en formato CSV.
# - `email.mime`: para crear mensajes de correo en formato multipart.

In [ ]:
# Carga de variables de entorno desde archivo .env
from dotenv import load_dotenv
import os

load_dotenv('Information.env')  # Carga las variables del archivo .env

email_remitente = os.getenv('EMAIL_REMITENTE')
contrasena_remitente = os.getenv('CONTRASENA_REMITENTE')

# Esta sección permite:
# - Cargar el correo y la contraseña del remitente desde un archivo `.env`.
# - Utiliza `os.getenv` para recuperar estas credenciales, que se usarán al enviar correos.

In [ ]:
# Función para asignar amigos invisibles
def asignar_amigos(participantes):
    nombres = list(participantes.keys())
    asignados = nombres[:]
    random.shuffle(asignados)
    
    resultado = {}
    for nombre in nombres:
        while asignados[0] == nombre:
            random.shuffle(asignados)
        resultado[nombre] = asignados.pop(0)
    
    return resultado

# Descripción:
# - Esta función toma un diccionario `participantes` (nombre: email) y asigna a cada participante un amigo invisible.
# - La asignación es aleatoria, y se asegura que ningún participante se asigne a sí mismo.
# - Devuelve un diccionario con las asignaciones.

In [ ]:
# Función para enviar correos electrónicos con las asignaciones
def enviar_correos(asignaciones, participantes, email_remitente, password_remitente):
    servidor = smtplib.SMTP('smtp.gmail.com', 587)
    servidor.starttls()
    servidor.login(email_remitente, password_remitente)
    
    for remitente, destinatario in asignaciones.items():
        correo = participantes[remitente]
        amigo_invisible = destinatario
        
        mensaje = MIMEMultipart()
        mensaje['From'] = email_remitente
        mensaje['To'] = correo
        mensaje['Subject'] = 'Tu amigo invisible'
        
        cuerpo = f'Hola {remitente}, tu amigo invisible es: {amigo_invisible}'
        mensaje.attach(MIMEText(cuerpo, 'plain'))
        
        servidor.send_message(mensaje)
        
    servidor.quit()

# Descripción:
# - Esta función envía un correo a cada participante con la asignación de su amigo invisible.
# - Configura el servidor SMTP, crea el mensaje personalizado para cada participante y lo envía.
# - Finalmente, cierra la conexión con el servidor de correo.


In [ ]:
# Función para imprimir en consola las asignaciones de amigos invisibles
def imprimir_asignaciones(asignaciones):
    print("Asignaciones de Amigos Invisibles:")
    for remitente, destinatario in asignaciones.items():
        print(f'{remitente} -> {destinatario}')

# Descripción:
# - Esta función imprime en consola las asignaciones de amigo invisible para cada participante.
# - Es útil para revisión rápida sin necesidad de enviar correos electrónicos.


In [ ]:
# Función principal que coordina la ejecución
def main():
    # Diccionario de participantes: 'nombre': 'correo'
    df_participantes = pd.read_csv('Name_Email.csv')
    participantes = dict(zip(df_participantes['Name'], df_participantes['Email']))
    
    # Credenciales del correo remitente
    email_remitente = os.getenv('EMAIL_REMITENTE')
    contrasena_remitente = os.getenv('CONTRASENA_REMITENTE')
    
    # Asignar amigos invisibles
    asignaciones = asignar_amigos(participantes)
    # Imprimir las asignaciones
    imprimir_asignaciones(asignaciones)
    # Enviar correos con las asignaciones
    enviar_correos(asignaciones, participantes, email_remitente, contrasena_remitente)

# Ejecutar el main
main()

# Descripción:
# - La función `main` es la función principal que organiza el flujo del programa.
# - Carga los participantes desde un archivo CSV (`Name_Email.csv`), realiza las asignaciones,
#   imprime las asignaciones y finalmente envía los correos con cada asignación.
